In [ ]:
# ===============================
# Imports and Constants
# ===============================
import os
import numpy as np
import pandas as pd
import polars as pl
from scipy.optimize import minimize, Bounds
from numba import njit
import kaggle_evaluation.default_inference_server

# Constants
MIN_POS, MAX_POS = 0, 2
TRADING_DAYS = 252

# Custom error
class ParticipantVisibleError(Exception):
    pass

# ===============================
# Numba-accelerated scoring
# ===============================
@njit(cache=True, fastmath=True)
def score_metric_jit(rf, fwd_ret, pos):
    n = len(pos)
    pos = np.clip(pos, MIN_POS, MAX_POS)

    # Strategy returns
    strat_ret = rf * (1.0 - pos) + fwd_ret * pos
    excess_ret = strat_ret - rf

    # Strategy stats
    strat_mean = np.power(np.prod(1.0 + excess_ret), 1.0 / n) - 1.0
    strat_std = np.std(strat_ret)
    if strat_std == 0:
        return 0.0

    sharpe = strat_mean / strat_std * np.sqrt(TRADING_DAYS)
    strat_vol = strat_std * np.sqrt(TRADING_DAYS) * 100.0

    # Market stats
    mkt_excess = fwd_ret - rf
    mkt_mean = np.power(np.prod(1.0 + mkt_excess), 1.0 / n) - 1.0
    mkt_std = np.std(fwd_ret)
    mkt_vol = mkt_std * np.sqrt(TRADING_DAYS) * 100.0

    # Volatility penalty
    vol_pen = 1.0
    if mkt_vol > 0.0:
        excess_vol = max(0.0, strat_vol / mkt_vol - 1.2)
        vol_pen = 1.0 + excess_vol

    # Return penalty
    ret_gap = max(0.0, (mkt_mean - strat_mean) * 100.0 * TRADING_DAYS)
    ret_pen = 1.0 + (ret_gap * ret_gap) / 100.0

    # Adjusted Sharpe
    adj_sharpe = sharpe / (vol_pen * ret_pen)
    return min(adj_sharpe, 1_000_000.0)


def score_metric(df: pd.DataFrame, preds: np.ndarray) -> float:
    rf = df["risk_free_rate"].to_numpy(np.float64)
    fwd_ret = df["forward_returns"].to_numpy(np.float64)
    return score_metric_jit(rf, fwd_ret, preds)

# ===============================
# Step 1: Precompute optimized weights (offline)
# ===============================
DATA_DIR = "/kaggle/input/hull-tactical-market-prediction/"
TRAIN_PATH = DATA_DIR + "train.csv"

# Load train data
df_train = pd.read_csv(TRAIN_PATH, index_col="date_id")
df_window = df_train.iloc[-180:].copy()

# Optimization objective
def objective_fn(weights: np.ndarray) -> float:
    try:
        return -score_metric(df_window, weights)
    except ParticipantVisibleError:
        return np.inf

# Run optimization
init_weights = np.full(180, 0.05, dtype=np.float64)
opt_result = minimize(
    objective_fn,
    init_weights,
    method="Powell",
    bounds=Bounds(lb=MIN_POS, ub=MAX_POS),
    options={"xtol": 1e-8, "ftol": 1e-8, "disp": True}
)
# Save weights to disk
np.save("opt_weights.npy", opt_result.x)

# ===============================
# Step 2: Kaggle inference server
# ===============================
# Load precomputed weights
opt_weights = np.load("opt_weights.npy")
pred_idx = 0

def predict(test_batch: pl.DataFrame) -> pl.DataFrame:
    """
    Predict batch for Kaggle DefaultInferenceServer.
    Returns Polars DataFrame with a single 'prediction' column.
    """
    global pred_idx, opt_weights
    n = test_batch.shape[0]
    preds = []

    for i in range(n):
        if pred_idx >= len(opt_weights):
            pred_idx = 0
        preds.append(float(opt_weights[pred_idx]))
        pred_idx += 1

    return pl.DataFrame({"prediction": preds})

# Initialize server
inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)

# Run server
if os.getenv("KAGGLE_IS_COMPETITION_RERUN"):
    inference_server.serve()
else:
    inference_server.run_local_gateway((DATA_DIR,))
